# Segmenting and Clustering Neighborhoods in Toronto - Problem#1

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [12]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [13]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [14]:
#dataframe will consists: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [15]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data 

### Data Cleaning

#### Ignore rows where Borough is 'Not assigned'

In [16]:
df=df[df['Borough']!='Not assigned']
df[df['Neighborhood']=='Not assigned']=df['Borough']

In [17]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [18]:
df_merge = pd.merge(df, temp_df, on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [19]:
df_merge.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto - Problem#2

In [20]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['Postalcode', 'Latitude', 'Longitude']
df_latlng.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Data frames on "Postalcode"

In [21]:
df_new = pd.merge(df_merge, df_latlng, on='Postalcode')
df_new.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
6,M3B,North York,Don Mills North,43.745906,-79.352188
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
9,M6B,North York,Glencairn,43.709577,-79.445073


# Segmenting and Clustering Neighborhoods in Toronto - Problem#3

### Import Toronto data

In [22]:
import json # library to handle JSON files

#conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

url="https://cocl.us/Geospatial_data"
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Get the URL

In [23]:
CLIENT_ID = 'PVLO1XLOLFHLK50CEARO2QPNTY1U0AP4HNYPJIIJODXAGWSS' # your Foursquare ID
CLIENT_SECRET = 'KK42LKUP0OSHVW1GB5JEWUDIOE5T3RSARPDNFIJJ3OLLSHHG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500
#print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: PVLO1XLOLFHLK50CEARO2QPNTY1U0AP4HNYPJIIJODXAGWSS
CLIENT_SECRET:KK42LKUP0OSHVW1GB5JEWUDIOE5T3RSARPDNFIJJ3OLLSHHG


'https://api.foursquare.com/v2/venues/search?client_id=PVLO1XLOLFHLK50CEARO2QPNTY1U0AP4HNYPJIIJODXAGWSS&client_secret=KK42LKUP0OSHVW1GB5JEWUDIOE5T3RSARPDNFIJJ3OLLSHHG&ll=43.653963,-79.387207&v=20180604&radius=500&limit=100'

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",False,4f04779a02d5cce0cfc06151,"393 University Ave, Suite 1501",CA,Toronto,Canada,in University Centre,14,"[393 University Ave, Suite 1501 (in University...","[{'label': 'display', 'lat': 43.65402839343005...",43.654028,-79.387365,NaN,NaN,ON,Chinese Visa Application Service Center,v-1581600408,NaN
1,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4c655857f67ec928e7485727,393 University Ave,CA,Toronto,Canada,NaN,79,"[393 University Ave, Toronto ON M5G 1E6, Canada]","[{'label': 'display', 'lat': 43.65375248805427...",43.653752,-79.388150,NaN,M5G 1E6,ON,393 University Ave,v-1581600408,NaN
2,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",False,4b66e19ff964a520922e2be3,393 University Avenue,CA,Toronto,Canada,NaN,49,"[393 University Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6535208759085,...",43.653521,-79.387193,NaN,NaN,ON,Ontario Superior Court of Justice,v-1581600408,NaN
3,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",False,4d8a3465d85f370415cbf8db,361 University Ave,CA,Toronto,Canada,at Queen St W,105,"[361 University Ave (at Queen St W), Toronto O...","[{'label': 'display', 'lat': 43.65376404494388...",43.653764,-79.385925,NaN,M5G 1T3,ON,Ontario Superior Court House,v-1581600408,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,52ce14b0498e50457ce11780,108 Chestnut Street,CA,Toronto,Canada,Dundas St W,124,"[108 Chestnut Street (Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.6546083, 'lng'...",43.654608,-79.385942,NaN,M5G 1R3,ON,DoubleTree by Hilton,v-1581600408,NaN


In [26]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",CA,Toronto,Canada,in University Centre,14,"[393 University Ave, Suite 1501 (in University...","[{'label': 'display', 'lat': 43.65402839343005...",43.654028,-79.387365,NaN,NaN,ON,4f04779a02d5cce0cfc06151
1,393 University Ave,Office,393 University Ave,CA,Toronto,Canada,NaN,79,"[393 University Ave, Toronto ON M5G 1E6, Canada]","[{'label': 'display', 'lat': 43.65375248805427...",43.653752,-79.388150,NaN,M5G 1E6,ON,4c655857f67ec928e7485727
2,Ontario Superior Court of Justice,Courthouse,393 University Avenue,CA,Toronto,Canada,NaN,49,"[393 University Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6535208759085,...",43.653521,-79.387193,NaN,NaN,ON,4b66e19ff964a520922e2be3
3,Ontario Superior Court House,Courthouse,361 University Ave,CA,Toronto,Canada,at Queen St W,105,"[361 University Ave (at Queen St W), Toronto O...","[{'label': 'display', 'lat': 43.65376404494388...",43.653764,-79.385925,NaN,M5G 1T3,ON,4d8a3465d85f370415cbf8db
4,DoubleTree by Hilton,Hotel,108 Chestnut Street,CA,Toronto,Canada,Dundas St W,124,"[108 Chestnut Street (Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.6546083, 'lng'...",43.654608,-79.385942,NaN,M5G 1R3,ON,52ce14b0498e50457ce11780


### Get the Venues in Toronto

In [31]:
#Create function to know how many venues there are in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get the Neighborhoods in Toronto by FourSquare API

In [32]:
#List of Neighborhood that have venues in Toronto
toronto_venues = getNearbyVenues(names=df_new['Neighborhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

toronto_venues.head()

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riverdale
Design

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [33]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))
toronto_venues.head()

2223 venues were returned by Foursquare.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### Group the Venues

In [34]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56


In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0


In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1       Steakhouse  0.04
2  Thai Restaurant  0.04
3              Bar  0.04
4             Café  0.04


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2               Skating Rink  0.25
3  Latin American Restaurant  0.25
4  Middle Eastern Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0  Grocery Store  0.18
1    Pizza Place  0.09
2    Coffee Shop  0.09
3       Pharmacy  0.09
4     Beer Store  0.09


----Alderwood, Long Branch----
          venue  fr

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
import numpy as np # library to handle data in a vectorized manner
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,Thai Restaurant,Bar
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Drugstore,Diner
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Discount Store,Pizza Place
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Gym,Pharmacy


#### Clustering the Neighborhoods

In [43]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged.dropna()


toronto_merged # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Eastern European Restaurant,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,French Restaurant,Pizza Place,Coffee Shop,Hockey Arena
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Café,Pub
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Event Space
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Gym,Park,Yoga Studio,Burrito Place
5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Dim Sum Restaurant,Falafel Restaurant,Fabric Shop,Event Space
6,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0.0,Fast Food Restaurant,Pizza Place,Gastropub,Pharmacy,Café
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop
9,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Sushi Restaurant,Pub,Japanese Restaurant,Dessert Shop


#### Create Cluster Map

In [48]:
# create map
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### List the venues by Cluster labels

In [50]:
toronto_merged.loc[toronto_merged['ClusterLabels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,1.0,Food & Drink Shop,Park,Eastern European Restaurant,Discount Store,Dog Run
20,York,1.0,Park,Women's Store,Market,Yoga Studio,Dumpling Restaurant
34,East York,1.0,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant
48,North York,1.0,Park,Bakery,Construction & Landscaping,Yoga Studio,Eastern European Restaurant
60,Central Toronto,1.0,Park,Swim School,Bus Line,Drugstore,Discount Store
65,North York,1.0,Park,Bank,Bar,Convenience Store,Eastern European Restaurant
84,Scarborough,1.0,Park,Playground,Yoga Studio,Drugstore,Diner
90,Downtown Toronto,1.0,Park,Trail,Playground,Dim Sum Restaurant,Diner
97,Etobicoke,1.0,River,Park,Drugstore,Dim Sum Restaurant,Diner
